In [1]:
import ee
import geemap
from geemap import ml
from sklearn import ensemble
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


# Inicializa la autenticación y la inicialización de Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-facuboladerasgee')

In [2]:
# Definir la región de interés
roi = ee.FeatureCollection('projects/ee-facuboladerasgee/assets/MalawiWoodfuelSite_1a')
# Centrar el mapa en la región de interés
Map = geemap.Map(center=[0, 0], zoom=2)
# Añadir Google Satellite como mapa base
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

In [3]:
roi_feature = ee.FeatureCollection(roi).first()

# Extraer la geometría de la característica
roi_geometry = roi_feature.geometry()

# Obtener las coordenadas de la geometría
roi_coordinates = roi_geometry.coordinates().getInfo()

# Imprimir las coordenadas para verificar
print("Coordenadas de ROI:", roi_coordinates)

# Guardar las coordenadas en una variable
roi_coords = roi_coordinates

ee_roi =  ee.Geometry.Polygon(roi_coords) 

Coordenadas de ROI: [[[35.147307857226416, -15.036316131901854], [35.15253608286859, -15.223952919778808], [35.15776440008984, -15.411589648079637], [35.20994926360607, -15.410686693842585], [35.26213417872549, -15.409783744326894], [35.314318994087046, -15.408880778443212], [35.366503893152895, -15.407977790870213], [35.3589679641149, -15.032891601485606], [35.30605294737285, -15.033747715996299], [35.25313786054075, -15.034603909691883], [35.20022286170109, -15.035460051719076], [35.147307857226416, -15.036316131901854]]]


In [13]:
# Definir el año de análisis
year = 2010

# Definir la fecha de inicio y fin para filtrar la colección
start = f'{year}-01-01'
end = f'{year}-12-31'

In [50]:
# s2_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
#     .filterBounds(roi) \
#     .filterDate(start, end) \
#     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

# cloud_col = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') \
#     .filterBounds(roi) \
#     .filterDate(start, end)

# def mask_clouds(image):
#     cloud_score = ee.Image(cloud_col.filterMetadata('system:index', 'equals', image.get('system:index')).first())
#     mask = cloud_score.select('probability').lt(10)
#     return image.updateMask(mask)

# # Aplicar máscara de nubes a la colección de Sentinel-2
# s2_masked = s2_col.map(mask_clouds)
# image = s2_masked.median().toFloat().clip(roi)

# # Calcular índices
# ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
# mndwi = image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
# ndbi = image.normalizedDifference(['B11', 'B8']).rename('NDBI')

# evi = image.expression(
#     '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
#     {
#         'NIR': image.select('B8'),
#         'RED': image.select('B4'),
#         'BLUE': image.select('B2')
#     }
# ).rename('EVI')

# savi = image.expression(
#     '((NIR - RED) / (NIR + RED + 0.5)) * 1.5',
#     {
#         'NIR': image.select('B8'),
#         'RED': image.select('B4')
#     }
# ).rename('SAVI')

# image = image.addBands([ndvi, mndwi, ndbi, evi, savi])

# # Cargar la colección Dynamic World y filtrar por la grilla y la fecha
# dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
#     .filterBounds(roi) \
#     .filterDate(start, end)

# # Añadir la banda de etiqueta de Dynamic World a la imagen
# dw_label = dw_col.select('label').median().clip(roi)
# image = image.addBands(dw_label.rename('label'))

# dem = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
# slope = ee.Terrain.slope(dem).rename('slope').clip(roi)

# image = image.addBands(dem.rename('elevation'))
# image = image.addBands(slope)

# # Cargar la imagen de altura del dosel y recortar a la grilla
# canopy_height = ee.Image("users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1").clip(roi)
# image = image.addBands(canopy_height.rename('CH'))

# def mask_edge(image):
#     edge = image.lt(-30.0)
#     masked_image = image.mask().And(edge.Not())
#     return image.updateMask(masked_image)

# ascending_filter = ee.Filter.eq('orbitProperties_pass', 'ASCENDING')
# descending_filter = ee.Filter.eq('orbitProperties_pass', 'DESCENDING')

# def create_filtered_collection(polarization):
#     return ee.ImageCollection('COPERNICUS/S1_GRD') \
#         .filterBounds(roi) \
#         .filterDate(start, end) \
#         .filter(ee.Filter.eq('instrumentMode', 'IW')) \
#         .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization)) \
#         .select(polarization) \
#         .map(mask_edge)

# img_vv_ascending = create_filtered_collection('VV').filter(ascending_filter)
# img_vh_ascending = create_filtered_collection('VH').filter(ascending_filter)
# img_vv_descending = create_filtered_collection('VV').filter(descending_filter)
# img_vh_descending = create_filtered_collection('VH').filter(descending_filter)

# def compute_median(image_collection, band_name):
#     if image_collection.size().getInfo() > 0:
#         return image_collection.median().rename(band_name)
#     else:
#         return ee.Image().rename(band_name)

# s1_vv_ascending = compute_median(img_vv_ascending, 'VV_Ascending')
# s1_vh_ascending = compute_median(img_vh_ascending, 'VH_Ascending')
# s1_vv_descending = compute_median(img_vv_descending, 'VV_Descending')
# s1_vh_descending = compute_median(img_vh_descending, 'VH_Descending')

# s1_combined = ee.Image.cat([s1_vv_ascending, s1_vh_ascending, s1_vv_descending, s1_vh_descending]).clip(roi)

# rvi_ascending = s1_combined.expression(
#     '4 * VH_Ascending / (VV_Ascending + VH_Ascending)',
#     {
#         'VV_Ascending': s1_combined.select('VV_Ascending'),
#         'VH_Ascending': s1_combined.select('VH_Ascending')
#     }
# ).rename('RVI_Ascending')

# rvi_descending = s1_combined.expression(
#     '4 * VH_Descending / (VV_Descending + VH_Descending)',
#     {
#         'VV_Descending': s1_combined.select('VV_Descending'),
#         'VH_Descending': s1_combined.select('VH_Descending')
#     }
# ).rename('RVI_Descending')

# image = image \
#     .addBands(s1_combined.select('VV_Ascending')) \
#     .addBands(s1_combined.select('VH_Ascending')) \
#     .addBands(s1_combined.select('VV_Descending')) \
#     .addBands(s1_combined.select('VH_Descending')) \
#     .addBands(rvi_ascending) \
#     .addBands(rvi_descending)
    


# bands = ['B11', 'B12','B8', 'B2', 'B3', 'B4', 'CH', 'MNDWI', 'NDBI', 'NDVI','EVI', 'SAVI','VV_Ascending', 'VH_Ascending', 'VV_Descending', 'VH_Descending','RVI_Ascending','RVI_Descending', 'elevation', 'slope','label' ]  
# image = image.select(bands)

In [17]:
# Cargar la colección de Landsat 7
l7_col = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
    .filterBounds(roi) \
    .filterDate(start, end) \
    .filter(ee.Filter.lt('CLOUD_COVER', 10))

def mask_clouds_landsat(image):
    # Seleccionar la banda de máscara de calidad (QA_PIXEL)
    qa = image.select('QA_PIXEL')

    # Crear una máscara para las nubes y sombras de nubes
    cloud_shadow_bit_mask = ee.Number(2).pow(3).int()
    clouds_bit_mask = ee.Number(2).pow(5).int()

    # Aplica las máscaras de nubes y sombras
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0) \
        .And(qa.bitwiseAnd(clouds_bit_mask).eq(0))

    # Aplicar la máscara de nubes y sombras
    return image.updateMask(mask)

# Aplicar la máscara de nubes a la colección de Landsat 7
l7_masked = l7_col.map(mask_clouds_landsat)
image = l7_masked.median().toFloat().clip(roi)

# Calcular índices
ndvi = image.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')
mndwi = image.normalizedDifference(['SR_B2', 'SR_B5']).rename('MNDWI')
ndbi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDBI')

evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
    {
        'NIR': image.select('SR_B4'),
        'RED': image.select('SR_B3'),
        'BLUE': image.select('SR_B1')
    }
).rename('EVI')

savi = image.expression(
    '((NIR - RED) / (NIR + RED + 0.5)) * 1.5',
    {
        'NIR': image.select('SR_B4'),
        'RED': image.select('SR_B3')
    }
).rename('SAVI')

image = image.addBands([ndvi, mndwi, ndbi, evi, savi])

# Cargar la colección Dynamic World y filtrar por la grilla y la fecha
dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterBounds(roi) \
    .filterDate('2020-01-01', '2020-12-01')

# Añadir la banda de etiqueta de Dynamic World a la imagen
dw_label = dw_col.select('label').median().clip(roi)
image = image.addBands(dw_label.rename('label'))

# Añadir DEM y pendiente
dem = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
slope = ee.Terrain.slope(dem).rename('slope').clip(roi)

image = image.addBands(dem.rename('elevation'))
image = image.addBands(slope)

# Funciones de preprocesamiento
def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

def radar_to_db_palsar(image, band):
    """
    Convierte los datos de la banda HH y HV de PALSAR según la fórmula dada.
    Machine learning feature importance selection for predicting aboveground biomass in African savannah with landsat 8 and ALOS PALSAR data Sa'ad Ibrahim
    """
    dn_squared = ee.Image(image).select(band).pow(2)
    backscatter = ee.Image(10).multiply(dn_squared.log10()).subtract(83.01).rename(band + '_dB')
    return backscatter

def preprocess(image):
    speckle_filtered = image.focal_median(kernelType='circle', radius=50, units='meters')
    return speckle_filtered


# Procesamiento de imágenes PALSAR
img_hh = (
    ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH')
    .filterBounds(roi)
    .filterDate(start, end)
    .select('HH')
    .map(preprocess)
)

img_hv = (
    ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH')
    .filterBounds(roi)
    .filterDate(start, end)
    .select('HV')
    .map(preprocess)
)

# Aplicar la conversión a dB
palsar_hh_db = img_hh.map(lambda img: radar_to_db_palsar(img, 'HH'))
palsar_hv_db = img_hv.map(lambda img: radar_to_db_palsar(img, 'HV'))

# Calcular la mediana
palsar_hh_median = palsar_hh_db.median().rename('HH_dB')
palsar_hv_median = palsar_hv_db.median().rename('HV_dB')

# Calcular el índice RVI
rvi = palsar_hv_median.divide(palsar_hh_median.add(palsar_hv_median)).multiply(4).rename('RVI_palsar')

# Combinar todas las bandas en una sola imagen
palsar_combined = ee.Image.cat([palsar_hh_median, palsar_hv_median, rvi]).clip(roi)

# Añadir las bandas combinadas a la imagen original
image = image.addBands(palsar_combined)

bands = ['SR_B1', 'SR_B2','SR_B3', 'SR_B4', 'SR_B5', 'MNDWI', 'NDBI', 'NDVI','EVI', 'SAVI','elevation', 'slope', 'HH_dB', 'HV_dB','RVI_palsar','label' ]  
image = image.select(bands)



In [18]:
feature_names = ['SR_B1', 'SR_B2','SR_B3', 'SR_B4', 'SR_B5', 'MNDWI', 'NDBI', 'NDVI','EVI', 'SAVI','elevation', 'slope', 'HH_dB', 'HV_dB','RVI_palsar','label' ]  
label = "agbd"

user_id = 'users/facuboladerasgee'
# specify asset id where to save trees
# be sure to change  to your ee user name
asset_id = user_id + "/Rf_Malawi2020-L-P"
asset_id

# read the exported tree feature collection
rf_fc = ee.FeatureCollection(asset_id)

# convert it to a classifier, very similar to the `ml.trees_to_classifier` function
another_classifier = ml.fc_to_classifier(rf_fc)

# classify the image again but with the classifier from the persisted trees
classified = image.select(feature_names).classify(another_classifier)

In [51]:
feature_names = ['B11', 'B12','B8', 'B2', 'B3', 'B4', 'CH', 'MNDWI', 'NDBI', 'NDVI','EVI', 'SAVI','VV_Ascending', 'VH_Ascending', 'VV_Descending', 'VH_Descending','RVI_Ascending','RVI_Descending', 'elevation', 'slope','label' ]  
label = "agbd"
user_id = 'users/facuboladerasgee'
def recombine_model_parts(user_id, base_asset_id, num_parts):
    parts = [ee.FeatureCollection(f"{base_asset_id}_part_{i}") for i in range(num_parts)]
    combined_fc = ee.FeatureCollection(parts[0])
    
    for part in parts[1:]:
        combined_fc = combined_fc.merge(part)
    
    return ml.fc_to_classifier(combined_fc)

# Ejemplo de uso:
num_parts = 2  # Ajusta este número según la cantidad de partes exportadas
base_asset_id = f"{user_id}/Rf_Malawi2020-S1-S2"
recombined_classifier = recombine_model_parts(user_id, base_asset_id, num_parts)

# Clasificar la imagen con el clasificador recombinado
classified = image.select(feature_names).classify(recombined_classifier)


In [14]:
# # Reducir la imagen clasificada a su valor máximo en la región de interés
# max_value = classified.reduceRegion(
#     reducer=ee.Reducer.max(),
#     geometry=roi,
#     scale=100,
#     maxPixels=1e9
# )

# # Obtener el valor máximo de la reducción
# max_agbd = max_value.get('classification').getInfo()
# print(f"Max value of the classified image: {max_agbd}")

Max value of the classified image: 155.93490694999983


In [19]:
# Centrar el mapa en la región de interés
Map = geemap.Map(center=[0, 0], zoom=2)
Map.centerObject(roi)

visPredictedBiomass = {
    'min': 0,
    'max': 200,
    'palette': ['lightyellow', 'lightgreen', 'green', 'darkgreen']
}

Map = geemap.Map(zoom=11)
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

Map.addLayer(
    classified,
    visPredictedBiomass,
    "classification",)
Map.add_colorbar(visPredictedBiomass, label="Predicted Biomass")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [20]:
import time

# Parámetros de exportación
task = ee.batch.Export.image.toDrive(
    image=classified,
    description='Malawi_2010-L-P',
    folder='Estimators_Malawi_complete',
    region=ee_roi,
    scale=30,
    crs='EPSG:4326',  # CRS adecuado para Nepal
    
    
)
task.start()

print("Exportación iniciada. Revisa tu Google Drive en la carpeta 'EE_RF'.")

# Esperar a que la tarea de exportación se complete (opcional)
while task.active():
    print('Exportación en progreso...')
    time.sleep(30)  # Esperar 30 segundos antes de verificar el estado nuevamente

if task.status()['state'] == 'COMPLETED':
    print('Exportación completada con éxito.')
else:
    print('Error en la exportación:', task.status())

Exportación iniciada. Revisa tu Google Drive en la carpeta 'EE_RF'.
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación completada con éxito.
